# Dropout and Strides

At the end of this lesson, you will understand and know how to use
- **Stride lengths** to make your model faster and reduce memory consumption
- **Dropout** to combat overfitting

Both of these techniques are especially useful in large models.

## Strides

Recall how we applied a convolution to each part of an image to get some filtered output.  You can imagine starting in the upper left, sliding rightwards one column at a time, and then repeating this for each row.

<img src="https://i.imgur.com/31OoSeu.gif">

But we could also slide across multiple columns at a time as we move across the image, and we could move down multiple rows at a time as we move down.  For example, if we moved across two columns at a time instead of one, we would apply each convolution half as many times, and the output from this layer would be half as wide.  For the same reason, moving down two rows at a time would make the output half as tall.  If we always moved in increments of two pixels, we would say the stride length is 2.  

<img src="https://i.imgur.com/18B2omZ.gif">

Since the resulting output was half as wide and half as tall, the representation going into the next layer ends up being only a quarter as large.  This makes the model much faster. You change the stride length in any `Conv2d` layer with the `strides` argument. Because the larger stride reduces computational demands, it might help you try using larger models with more layers or more convolutions per layer. 

There are alternative ways to achieve the same effect as strides, including something called max-pooling.  But strides are conceptually cleaner than the alternatives, and there doesn’t seem to be a systematic difference in model performance for most applications.  For a few advanced applications like generative models, increasing stride length as you’ve seen here works much better than max-pooling.

## Dropout

The other technique I’ll show you is called dropout.  With dropout, we ignore randomly chosen nodes or convolutions for brief parts of training, and then randomly pick other nodes to ignore for the next part of training.  Most people are surprised when they first hear this can help. But it prevents overfitting. Specifically, it makes each convolution or node find information that is useful for prediction in its own right, rather than allowing one node to dominate the prediction, with everything else being small tweaks on that one node’s conclusion.

To add dropout to our network, we add a `Dropout` layer to our `Sequential` model. For example, `model.add(Dropout(0.5))` says that each connection in the preceding layer should be ignored or disconnected from the subsequent layer 50% of the time during model training. During inference, when we're making predictions, all nodes and all connections are used.

Before we knew about dropout, researchers addressed overfitting by limiting model capacity.  That meant they had few layers and fewer nodes or convolutions per layer.  But it’s common now to build large networks that could easily overfit, and then address overfitting by adding dropout.

The code on this page gives an example using strides of two and dropout.  You’ll use these techniques and tune the number of layers in the exercise. Both of these techniques will make you more effective with larger networks, which is a big step up in your deep learning capabilities.

In [ ]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout

img_rows, img_cols = 28, 28
num_classes = 10

def data_prep(raw):
    out_y = keras.utils.to_categorical(raw.label, num_classes)

    num_images = raw.shape[0]
    x_as_array = raw.values[:,1:]
    x_shaped_array = x_as_array.reshape(num_images, img_rows, img_cols, 1)
    out_x = x_shaped_array / 255
    return out_x, out_y

train_size = 30000
train_file = "../input/digit-recognizer/train.csv"
raw_data = pd.read_csv(train_file)

x, y = data_prep(raw_data)

model = Sequential()
model.add(Conv2D(30, kernel_size=(3, 3),
                 strides=2,
                 activation='relu',
                 input_shape=(img_rows, img_cols, 1)))
model.add(Dropout(0.5))
model.add(Conv2D(30, kernel_size=(3, 3), strides=2, activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x, y,
          batch_size=128,
          epochs=2,
          validation_split = 0.2)

# Exercise
**[Apply dropout and strides](#$EXERCISE_FORKING_URL$)** yourself while experimenting with larger models.